In [ ]:
!pip install slack-sdk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.5/292.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
import json
from pydantic import BaseModel
from slack_sdk.webhook import WebhookClient
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
from openai import OpenAI

client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key="Your_API_Key")

SLACK_WEBHOOK_URL = "WEBHOOK_URL"
slack_client = WebhookClient(SLACK_WEBHOOK_URL)

class Sentiment(BaseModel):
    sentiment: str
    responsible_area: str

def send_slack_alert(feedback, sentiment, areas_of_concern):
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        slack_message = f"""
        🚨 *Negative Feedback Alert* 🚨
        *Timestamp:* {timestamp}
        *Feedback:* {feedback}
        *Sentiment:* {sentiment.upper()}
        *Areas of Concern:* {', '.join(areas_of_concern)}
        """
        slack_client.send(text=slack_message)
    except Exception as e:
        print(f"Error sending Slack alert: {e}")

def provide_sentiment(feedback: str):
    try:
        print("🔍 Analyzing feedback...")
        completion = client.chat.completions.create(
            temperature=0,
            model="google/learnlm-1.5-pro-experimental:free",
            messages=[
                {"role": "system", "content": """You are an assistant that analyzes hotel guest feedback and provides sentiment analysis in JSON format. The sentiment should be Positive, Negative, or Neutral. If the sentiment is Negative, identify the specific area responsible for the feedback, including one or more of the following categories: 'Room Quality', 'Cleanliness', 'Staff Service', 'Food & Beverage', 'Amenities', 'Check-in/Check-out Process', 'Location', 'Pricing', 'Noise', 'Safety', 'Guest Room Comfort', 'Bathroom Facilities', 'Hotel Ambience', 'Parking', 'Eco-friendliness', 'Pet Policy', 'Business Facilities', 'Spa & Wellness', and 'Other'. If no specific area applies, return 'N/A'. Ensure that responses are clear and helpful."""},
                {"role": "user", "content": f"Here is a feedback for analysis: \"{feedback}\""}
            ]
        )
        sentiment_response = completion['choices'][0]['message']['content']
        sentiment_data = json.loads(sentiment_response)

        sentiment = sentiment_data.get("sentiment", "Error")
        areas_of_concern = sentiment_data.get("responsible_area", "N/A").split(", ")

        print("✅ Analysis complete!")
        print("\n**Sentiment Analysis Result:**")
        print(f"Sentiment: {sentiment.upper()}")
        print(f"Areas of Concern: {', '.join(areas_of_concern)}")

        if sentiment.lower() == "negative":
            send_slack_alert(feedback, sentiment, areas_of_concern)

    except Exception as e:
        print(f"Error occurred: {e}")
        print("\n**Sentiment Analysis Result:**")
        print("Sentiment: ERROR")
        print("Areas of Concern: N/A")

feedback_input = widgets.Textarea(
    value='',
    placeholder='Enter hotel guest feedback here...',
    description='Feedback:',
    layout=widgets.Layout(width='50%', height='100px')
)

analyze_button = widgets.Button(
    description='Analyze Feedback',
    button_style='primary',
    tooltip='Click to analyze the feedback',
    icon='search'
)

output_label = widgets.Label(value="Sentiment Analysis Result:")
output_area = widgets.Output()

def on_analyze_button_click(b):
    feedback = feedback_input.value
    with output_area:
        output_area.clear_output()
        provide_sentiment(feedback)

analyze_button.on_click(on_analyze_button_click)

display(feedback_input, analyze_button, output_label, output_area)
